In [ ]:
import boto3
import json
import pprint
import pandas as pd

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('config.cfg'))

KEY                = config.get("AWS_CONNECT","KEY")
SECRET             = config.get("AWS_CONNECT","SECRET")
                                
CLUSTER_TYPE       = config.get("SET_CONFIG","CLUSTER_TYPE")
NUM_NODES          = config.get("SET_CONFIG","NUM_NODES")
NODE_TYPE          = config.get("SET_CONFIG","NODE_TYPE")
CLUSTER_IDENTIFIER = config.get("SET_CONFIG","CLUSTER_IDENTIFIER")
DB                 = config.get("SET_CONFIG","DB")
USER               = config.get("SET_CONFIG","USER")
PASSWORD           = config.get("SET_CONFIG","PASSWORD")
PORT               = config.get("SET_CONFIG","PORT")
IAM_ROLE_NAME      = config.get("SET_CONFIG","IAM_ROLE_NAME")
                                
S3_NAME            = config.get("S3_CONFIG","S3_NAME")
S3_LOG_DATA_NAME   = config.get("S3_CONFIG","S3_LOG_DATA_NAME")
S3_SONG_DATA_NAME  = config.get("S3_CONFIG","S3_SONG_DATA_NAME")

pd.DataFrame(
    {
        "Param":[
            "CLUSTER_TYPE", 
            "NUM_NODES", 
            "NODE_TYPE", 
            "CLUSTER_IDENTIFIER", 
            "DB", 
            "DB_USER", 
            "DB_PASSWORD", 
            "PORT", 
            "IAM_ROLE_NAME",
            'S3_NAME',
            'S3_LOG_DATA_NAME',
            'S3_SONG_DATA_NAME',
            ],
         "Value":[
            CLUSTER_TYPE, 
            NUM_NODES, 
            NODE_TYPE, 
            CLUSTER_IDENTIFIER, 
            DB, 
            USER, 
            PASSWORD, 
            PORT, 
            IAM_ROLE_NAME,
            S3_NAME,
            S3_LOG_DATA_NAME,
            S3_SONG_DATA_NAME,
            ]
    }
)

## Create clients for EC2, S3, IAM, and Redshift

In [ ]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## Check out some log-data and song-data samples in S3 bucket


In [ ]:
bucket=s3.Bucket(S3_NAME)
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix=S3_LOG_DATA_NAME)]
song_data_files = [filename.key for filename in bucket.objects.filter(Prefix=S3_SONG_DATA_NAME+'/A/A')]

# print the last five files
print(f'Print the last 5 json /log-data* files in S3 bucket: {S3_NAME}')
pprint.pprint(log_data_files[-5:])
print(f'\nPrint the last 5 json /song-data* files in S3 bucket: {S3_NAME}')
pprint.pprint(song_data_files[-5:])

## Creating a new IAM-Role

In [ ]:
from botocore.exceptions import ClientError

# Create the role, 
try:
    print("Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
             'Effect': 'Allow',
             'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
print("Attaching Policy")
iam.attach_role_policy(
    RoleName=IAM_ROLE_NAME,
    PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    )['ResponseMetadata']['HTTPStatusCode']

print("Get the IAM role ARN")
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

## Creating a new cluster

In [ ]:
try:
    response = redshift.create_cluster(        
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        # Identifiers & Credentials
        DBName=DB,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=USER,
        MasterUserPassword=PASSWORD,
        
        # Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

## Describing the cluster

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = [
        "ClusterIdentifier", 
        "NodeType", 
        "ClusterStatus", 
        "MasterUsername", 
        "DBName", 
        "Endpoint", 
        "NumberOfNodes", 
        "VpcId",
    ]
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
ENDPOINT = myClusterProps['Endpoint']['Address']
print(ENDPOINT)
ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print(ROLE_ARN)

## Open an incoming TCP port to access the cluster ednpoint

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(PORT),
        ToPort=int(PORT)
    )
except Exception as e:
    print(e)

In [ ]:
%reload_ext sql

In [ ]:
conn_string=f"postgresql://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DB}"
print(conn_string)
%sql $conn_string

## Stop here and run 
> `python etl.py`

# Cheking Results

In [ ]:
%sql SELECT * FROM dim_users LIMIT 5;

In [ ]:
%sql SELECT * FROM dim_songs LIMIT 5;

In [ ]:
%sql SELECT * FROM dim_users LIMIT 5;

In [ ]:
%sql SELECT * FROM dim_artists LIMIT 5;

In [ ]:
%sql SELECT * FROM dim_time LIMIT 5;

In [ ]:
%sql SELECT * FROM fact_songplays LIMIT 5;

## Delete Cluster and IAM role 

In [ ]:
redshift.delete_cluster( ClusterIdentifier=CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=IAM_ROLE_NAME)